In [1]:
from os import listdir
from matplotlib import pyplot as pl
import pandas as pd
import numpy as np

In [2]:
path = "./selected_raw_data/"

In [3]:
def slice_generation(data):
    n_samples_per_slice = 10
    slice_overlap = 5

    storage = {}
    for f in data:
        # skip lud data, for the moment we only consider one person
        if 'lud' in f:
            continue

        # sort data in the python dict according to type of exercise (found in filename)    
        if 'corde' in f:
            label = 'corde'
        elif 'squats' in f:
            label = 'squats'
        elif 'dorsaux' in f:
            label = 'dorsaux'
        elif 'abdos' in f:
            label = 'abdos'

        # iterate through the file to generate slices
        # by starting at "slice_duration", we skip the first slice, which might not be relevant
        # we also drop the last slice by stopping the loop 2*slice duration before the end
        for start_sample in range(n_samples_per_slice, len(data[f]['timestamp'])-2*n_samples_per_slice, slice_overlap):
            df_temp = data[f].iloc[start_sample:start_sample+n_samples_per_slice, :]
            df_temp = df_temp.drop(columns='timestamp')

            # TODO: drop first and last slice of each file

            if label in storage.keys():          # if the label already exists in the dictionary
                temp = storage[label]
                temp.append(df_temp)           # append the new geture to the list
                storage[label] = temp
            else:
                storage[label] = [df_temp,] # otherwise, create a list with one element
                
    return storage

## Import data
Steps for each file :
- print filename (useful to access data later)
- create a dataframe from the values inside the file
- clean timestamp in order to get a column indicating the elapsed time in seconds

In [4]:
data_test ={}
data_train ={}
for f in listdir(path):
    if "valid" in f:
        print("Data for validation : ",f)
        data_test[f] = pd.read_csv(path+f)
        start_time = data_test[f]["timestamp"][0]
        data_test[f]["timestamp"] = (data_test[f]["timestamp"]-start_time)/1000
    else :
        print("Data for training : ",f)
        data_train[f] = pd.read_csv(path+f)
        start_time = data_train[f]["timestamp"][0]
        data_train[f]["timestamp"] = (data_train[f]["timestamp"]-start_time)/1000

Data for training :  abdos_f5_lud.csv
Data for training :  abdos_f5_manon1.csv
Data for training :  abdos_f5_manon2.csv
Data for training :  abdos_f5_manon3.csv
Data for validation :  abdos_f5_manon4_valid.csv
Data for training :  corde_f5_manon1.csv
Data for training :  corde_f5_manon2.csv
Data for training :  corde_f5_manon3.csv
Data for validation :  corde_f5_manon4_valid.csv
Data for training :  dorsaux_f5_lud.csv
Data for training :  dorsaux_f5_manon1.csv
Data for training :  dorsaux_f5_manon2.csv
Data for training :  dorsaux_f5_manon3.csv
Data for validation :  dorsaux_f5_manon4_valid.csv
Data for training :  squats_f5_lud.csv
Data for training :  squats_f5_manon1.csv
Data for training :  squats_f5_manon2.csv
Data for training :  squats_f5_manon3.csv
Data for validation :  squats_f5_manon4_valid.csv


##  Plots
### Abdos

In [6]:
data = data_test + data_train

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [5]:
for f in data:
    if "abdos" in f:
        data[f].plot(x="timestamp", y=['x', 'y', 'z'], kind='line', title=f, grid=True)
        pl.ylabel('acceleration [N/kg]')
        pl.xlabel('time [s]')

NameError: name 'data' is not defined